In [2]:
Pkg.add("MultivariateStats")

INFO: Nothing to be done


In [3]:
using MultivariateStats

INFO: Recompiling stale cache file C:\Users\Zacharias\.julia\lib\v0.4\MultivariateStats.ji for module MultivariateStats.


In [6]:
Pkg.update()

INFO: Updating METADATA...
INFO: Updating GeneratedTypes...
INFO: Updating GeneratedTables...
INFO: Updating TSne...
INFO: Updating GraphPlot...
INFO: Updating ComputeFramework...
INFO: Updating ELM...
INFO: Updating PlotlyJS...
INFO: Updating ROC...
INFO: Computing changes...
INFO: No packages to install, update or remove


In [5]:
cd("d:\\data\\OnlineNewsPopularity")

In [35]:
X = readcsv("OnlineNewsPopularity.csv")[2:end,2:end] # get the numeric data only

N, n = size(X)
n -= 1 # remove output column in the col. count (so, now n = number of features)
I = Array(Float64, N, n)
O = X[:, end]

for j = 1:n
    for i = 1:N
        I[i,j] = Float64(X[i,j])
    end
end

In [37]:
ind = (O .>= 10000) # cases of "viral" result
O_bin = falses(N)
O_bin[ind] = true

true

In [41]:
z = round(Int64, N*0.8)
ind = randperm(N)

39644-element Array{Int64,1}:
  4441
 11863
 37218
  1432
 14723
 15635
 19759
 27440
  8768
 16895
 10879
 18196
  3080
     ⋮
  9457
  4924
 16534
  6600
   404
 19262
  8012
  7377
  2288
  7535
  6210
 37897

In [42]:
Itr = I[ind[1:z],:]
Ite = I[ind[(z+1):end],:]
Otr = O[ind[1:z]] # this will be useful later on
Ote = O[ind[(z+1):end]] # this will be useful later on

7929-element Array{Any,1}:
  1800
  1100
   928
   860
  4900
 14100
  2400
   804
  2300
  5000
  1400
  1400
   906
     ⋮
  1200
  1600
  4100
  1200
  1300
  2900
    99
  7600
  1200
 21500
  4600
  4800

In [43]:
M = fit(PCA, Itr'; maxoutdim = 10) # limit the total number of variables in reduced feature set to 10

PCA(indim = 59, outdim = 4, principalratio = 0.99432)

In [44]:
Jtr = transform(M, Itr')'

31715x4 Array{Float64,2}:
     -1.72968e5   -51330.0         10100.2    -2609.47
  40486.2        -125950.0         23542.6    -1036.78
      1.0455e5      5030.53       -15348.9   -15228.2 
     -7.17541e5    79428.4         -6427.05   -4027.68
  45391.4             -1.15521e5    8467.82   41557.6 
  51924.2        -102180.0         16139.9     3151.56
 107273.0          10448.0        -25476.0     9492.71
      1.19281e5    35401.8        -22503.3   -19913.1 
 -11037.3              1.16436e5  -35440.1   -26302.9 
  70928.3         -63497.3          6659.22  -10030.9 
  11265.4        -185433.0         41757.2     6611.88
 118538.0          44346.8         26455.4    -8769.9 
     -5.6765e5     27254.1          3706.9    -5296.01
      ⋮                                               
     -1.11233e5   -89426.2         19023.9    -1276.48
      1.1379e5     23576.1        -21765.4   -19956.7 
      1.32817e5    63813.4        -26433.3   -24753.4 
      1.08093e5    11968.8        -1855

In [45]:
Jte = transform(M, Ite')'

7929x4 Array{Float64,2}:
       1.49575e5   97375.6        -42102.4     -21798.3  
       1.05878e5    8668.34       -10786.6     -18035.9  
   52024.4            -1.01127e5   20819.2       1477.77 
 -118183.0            -1.03591e5   23247.6        825.798
  -43732.7         49823.5        -21841.5       4234.47 
       1.19078e5   34395.4        -26970.7     -14412.6  
   38268.0            -1.2926e5     -126.416    96671.1  
       1.05539e5    6948.36       -15770.1     -17636.2  
       1.09631e5   15109.6        -19560.2     -17524.9  
       1.14631e5   25293.4        -22443.0     -19602.6  
   78405.0        -47887.7          1318.78     -3894.65 
   81218.3        -42824.6         -1748.13    -10940.4  
   70644.2        -64367.7          4259.69     -6243.71 
       ⋮                                                 
   65923.8        -69799.5          6402.4      63307.2  
      -1.52232e5   -8922.52        -1099.67    -11099.8  
   46100.7            -1.10885e5   30151.6     

In [14]:
function cossim(x::Array, y::Array)
    # Cosine Similarity Function
    nx = norm(x)
    ny = norm(y)
    
    if (nx == 0) | (ny == 0)
        return NaN
    else
        return dot(x,y) / (nx * ny)
    end
end

cossim (generic function with 1 method)

In [46]:
V = Array(Float64, n);

In [47]:
for i = 1:n
    V[i] = cossim(Itr[:,i], Otr)
end

In [48]:
abs(V[1:5]), mean(abs(V))

([0.25362064598653405,0.28619684676702817,0.21589821404321854,0.041548246519832646,0.04977053431283567],0.17628312528531498)

In [49]:
th = 0.2
feat_ind = (abs(V) .>= th)
sum(feat_ind)

26

In [50]:
show(find(feat_ind)) # features in reduced feature set

[1,2,3,7,11,12,23,24,25,26,27,42,44,45,46,47,48,49,50,51,52,53,54,55,56,58]

In [51]:
Otr = Array(O_bin[ind[1:z]])
Ote = Array(O_bin[ind[(z+1):end]])

7929-element Array{Bool,1}:
 false
 false
 false
 false
 false
  true
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
  true
 false
 false

In [52]:
include("C:\\users\\Zacharias\\Documents\\Julia\\DID.jl")

DID (generic function with 4 methods)

In [53]:
for i = 1:n
    V[i] = DID(Itr[:,i], Otr)[1]
end

In [54]:
V[1:5], mean(V)

([0.50965273498501,0.19815453812576508,0.040647320638097506,0.1667844945508427,0.07425604603497078],0.19631837816413597)

In [55]:
th = 0.35
feat_ind = (V .>= th)
sum(feat_ind)

10

In [56]:
show(find(feat_ind)) # features in reduced feature set

[1,14,25,32,33,34,42,54,56,58]